In [19]:
import correctionlib
import hist
import awkward as ak
import numpy as np

from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea.lookup_tools.correctionlib_wrapper import correctionlib_wrapper
from coffea.lookup_tools.dense_lookup import dense_lookup

import sys
sys.path.append("../boostedhiggs/")

import btag
from btag import btagWPs

In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Load a root file

In [21]:
! ls ../rootfiles/

HWW   QCD   TTbar WJets


In [22]:
events = NanoEventsFactory.from_root(
    "../../boostedhiggs/rootfiles/HWW/file1.root",
    entry_stop=100_000,
    schemaclass=NanoAODSchema,
).events()

/Users/fmokhtar/miniforge3/envs/coffea-env/lib/python3.9/site-packages/coffea/nanoevents/mapping/uproot.py:86: UserWarning: Found duplicate branch FatJetAK15SubJet_nBHadrons in <TTree 'Events' (2140 branches) at 0x0001070fbe80>, taking first instance
  warnings.warn(
/Users/fmokhtar/miniforge3/envs/coffea-env/lib/python3.9/site-packages/coffea/nanoevents/mapping/uproot.py:86: UserWarning: Found duplicate branch FatJetAK15SubJet_nCHadrons in <TTree 'Events' (2140 branches) at 0x0001070fbe80>, taking first instance
  warnings.warn(
/Users/fmokhtar/miniforge3/envs/coffea-env/lib/python3.9/site-packages/coffea/nanoevents/mapping/uproot.py:86: UserWarning: Found duplicate branch FatJetAK15_nBHadrons in <TTree 'Events' (2140 branches) at 0x0001070fbe80>, taking first instance
  warnings.warn(
/Users/fmokhtar/miniforge3/envs/coffea-env/lib/python3.9/site-packages/coffea/nanoevents/mapping/uproot.py:86: UserWarning: Found duplicate branch FatJetAK15_nCHadrons in <TTree 'Events' (2140 branches)

# Compute efficiencies

In [66]:
year = "2017"
wp = "L"

# define your good jets
goodjets = events.Jet[
            (events.Jet.pt > 30) & (abs(events.Jet.eta) < 5.0) & events.Jet.isTight & (events.Jet.puId > 0)
        ]
# reject EE noisy jets for 2017
if year == "2017":
    goodjets = goodjets[(goodjets.pt > 50) | (abs(goodjets.eta) < 2.65) | (abs(goodjets.eta) > 3.139)]

jets = ak.flatten(goodjets)

# get efficiencies
efficiencyinfo = (
    hist.Hist.new
    .Reg(20, 40, 300, name="pt")
    .Reg(4, 0, 2.5, name="abseta")
    .IntCat([0, 4, 5], name="flavor")
    .Bool(name="passWP")
    .Double()
    .fill(
        pt=jets.pt,
        abseta=abs(jets.eta),
        flavor=jets.hadronFlavour,
        passWP=jets.btagDeepFlavB > btagWPs["deepJet"][year][wp], # UL 2018 medium WP
    )
)
efficiencyinfo

Hist(
  Regular(20, 40, 300, name='pt'),
  Regular(4, 0, 2.5, name='abseta'),
  IntCategory([0, 4, 5], name='flavor'),
  Boolean(name='passWP'),
  storage=Double()) # Sum: 48122.0 (63042.0 with flow)

In [72]:
eff = efficiencyinfo[{"passWP": True}] / efficiencyinfo[{"passWP": sum}]
# note this seems to turn 0,4,5 into 0,1,2
efflookup = dense_lookup(eff.values(), [ax.edges for ax in eff.axes])
efflookup

3 dimensional histogram with axes:
	1: [ 40.  53.  66.  79.  92. 105. 118. 131. 144. 157. 170. 183. 196. 209.
 222. 235. 248. 261. 274. 287. 300.]
	2: [0.    0.625 1.25  1.875 2.5  ]
	3: [0. 1. 2. 3.]

In [79]:
eff

Hist(
  Regular(20, 40, 300, name='pt'),
  Regular(4, 0, 2.5, name='abseta'),
  IntCategory([0, 4, 5], name='flavor'),
  storage=Double()) # Sum: nan (nan with flow)

In [73]:
efflookup

3 dimensional histogram with axes:
	1: [ 40.  53.  66.  79.  92. 105. 118. 131. 144. 157. 170. 183. 196. 209.
 222. 235. 248. 261. 274. 287. 300.]
	2: [0.    0.625 1.25  1.875 2.5  ]
	3: [0. 1. 2. 3.]

In [74]:
# Efficiency at 42 GeV, |eta|=0.2, for light, c, and b quark respectively
efflookup(np.array([42,60]), 0.2, 2)

array([0.86046512, 0.93103448])